# **Medication Correction**

## Step 2 - Mapping Standardized Drug Names to AHFS Therapeutic Classes 

by Chen Chen (c.chen2@wustl.edu); David Brown (browndavid@wustl.edu)
20250721

# Import libraries and Input data

In [1]:
# General
import datetime
import os
from timeit import default_timer as timer # Preferred across OS; see https://stackoverflow.com/a/25823885

# Data handling
import numpy as np
import pandas as pd

# Data visualization
import matplotlib.pyplot as plt

# import theFuzz package
from thefuzz import process
from thefuzz import fuzz

# Useful
import ast
import re
import string

In [2]:
# Overall execution duration
analysis_start = timer()

In [4]:
fp_cwd = os.getcwd() + "/"

# Input RedCap data after DER
input_data = pd.read_csv(fp_cwd + "20250715_drugs_aft_DER.csv", low_memory=False)

# Input AHFS data
input_ahfs_class = pd.read_csv(fp_cwd + "DrugBank_AHFS/AHFS_class.csv", low_memory=False)
input_ahfs_drug = pd.read_csv(fp_cwd + "DrugBank_AHFS/AHFS_drug.csv", low_memory=False)
input_ahfs_trade_name = pd.read_csv(fp_cwd + "DrugBank_AHFS/AHFS_trade_name.csv", low_memory=False)
input_ahfs_scdf = pd.read_csv(fp_cwd + "DrugBank_AHFS/AHFS_SCDF.csv", low_memory=False)
ndc_scdf_un = pd.read_csv(fp_cwd + "DrugBank_AHFS/NDC_SCDF_UN.csv", low_memory=False)
class_extend = pd.read_csv(fp_cwd + "DrugBank_AHFS/AHFS_class_extend.csv", low_memory=False)

# Functions

In [5]:
# Function to convert the Drug names to lower case
def my_lower (df, col):
    df[col]=df[col].str.lower().str.strip()
    return df

# Preprocessing

In [6]:
# Convert drug names that need to use as merge key to lower case
df_med = my_lower(input_data, 'DrugName').dropna()
print(f'Shape of Input # of Drugs:{df_med.shape}')

df_ahfs_drug = my_lower(input_ahfs_drug, 'GenDrugName')
print(f'Shape of AHFS Drugs:{df_ahfs_drug.shape}')

df_ahfs_trade_name = my_lower(input_ahfs_trade_name, 'Tradename')
print(f'Shape of AHFS Trade Names:{df_ahfs_trade_name.shape}')

df_ahfs_class_scdf = my_lower(input_ahfs_scdf, 'AHFSClassText')
print(f'Shape of AHFS Class SCDF:{df_ahfs_class_scdf.shape}')

Shape of Input # of Drugs:(15158, 7)
Shape of AHFS Drugs:(7068, 4)
Shape of AHFS Trade Names:(10649, 3)
Shape of AHFS Class SCDF:(12688, 4)


# Match DER output to AHFSDrugs

In [8]:
input_data[['DrugName']].nunique()

DrugName    552
dtype: int64

In [9]:
input_data[['map_id', 'otdate', 'Source', 'Number', 'MedName', 'DrugName', 'DrugbankId']].drop_duplicates().shape

(16961, 7)

In [10]:
# Left join 'DrugName' to tblAHFSDrugs
df_med_ahfs_drug = pd.merge(df_med, df_ahfs_drug, 
                            left_on='DrugName', 
                            right_on='GenDrugName', 
                            how='left').drop_duplicates()
df_med_ahfs_drug.shape

(27050, 11)

In [11]:
# Sanity check
check_N_df_med_ahfs_drug = df_med_ahfs_drug[['map_id', 'otdate', 'Source', 'Number', 'MedName', 'DrugName', 'DrugbankId']].drop_duplicates()
check_N_df_med_ahfs_drug.shape

(15158, 7)

In [12]:
# Matched Drugs
matched_ahfs_drug = df_med_ahfs_drug.dropna()
print(f'Shape of matched_ahfs_drug: {matched_ahfs_drug.shape}')

Shape of matched_ahfs_drug: (26224, 11)


In [13]:
df_med_ahfs_drug[['GenDrugName']].nunique()

GenDrugName    389
dtype: int64

In [14]:
# No matched Drugs
no_match_ahfs_drug = df_med_ahfs_drug[df_med_ahfs_drug.isna().any(axis=1)]
print(f'Shape of no_match_ahfs_drug: {no_match_ahfs_drug.shape}')
print(f'Number of unique DrugName: {no_match_ahfs_drug['DrugName'].nunique()}')

Shape of no_match_ahfs_drug: (826, 11)
Number of unique DrugName: 88


# Match DER output to AHFS Trade name

In [16]:
# Left join 'DrugName' to tblAHFSTradeName 
df_med_ahfs_drug_tradename = pd.merge(
    no_match_ahfs_drug,
    df_ahfs_trade_name,
    left_on="DrugName",
    right_on="Tradename",
    how="left").drop_duplicates()
print(f'shape of df_med_ahfs_drug_tradename: {df_med_ahfs_drug_tradename.shape}')

# Fill missing 'UN' with matched tradename 'UN'
df_med_ahfs_drug_tradename['UN_x'] = df_med_ahfs_drug_tradename['UN_x'].fillna(df_med_ahfs_drug_tradename['UN_y'])

# Check the missing value in df_med_ahfs_drug_tradename
df_med_ahfs_drug_tradename.isna().sum()

shape of df_med_ahfs_drug_tradename: (829, 14)


map_id           0
otdate           0
Source           0
Number           0
MedName          0
DrugName         0
DrugbankId       0
GenDrugName    829
UN_x           551
AHFSClassID    829
PrintClass     829
UN_y           551
Tradename      551
Type           551
dtype: int64

In [18]:
# Matched Drugs
matched_ahfs_trade_name = df_med_ahfs_drug_tradename.dropna(subset=['UN_x'])
print(f'Shape of matched_ahfs_trade_name: {matched_ahfs_trade_name.shape}')

Shape of matched_ahfs_trade_name: (278, 14)


In [19]:
print(f'Number of unique drugs in matched_ahfs_trade_name: {matched_ahfs_trade_name['DrugName'].nunique()}')

Number of unique drugs in matched_ahfs_trade_name: 31


In [20]:
# Extract no match with trade names
no_match_trade_name = df_med_ahfs_drug_tradename[df_med_ahfs_drug_tradename['UN_x'].isna()]
print(f'Shape of no_match_trade_name: {no_match_trade_name.shape}')
print(f'Number of unique drugs in no_match_trade_name: {no_match_trade_name['DrugName'].nunique()}')

Shape of no_match_trade_name: (551, 14)
Number of unique drugs in no_match_trade_name: 57


# Fuzzy Match using theFuzz package

In [21]:
%%time
# Extract one best match SCDF_STR to RedCapMedName
best_drug = lambda x: process.extractOne(x, df_ahfs_class_scdf["SCDF_STR"])
no_match_trade_name = no_match_trade_name.copy()

# Store the full tuple
no_match_trade_name["MatchTuple"] = no_match_trade_name["MedName"].map(best_drug)

# Extract the "Best_Match", "Similarity_Score", and "Best_Match_Index" from the full tuple
no_match_trade_name["Best_Match"] = no_match_trade_name["MatchTuple"].apply(lambda x: x[0] if x else np.nan)
no_match_trade_name["Similarity_Score"] = no_match_trade_name["MatchTuple"].apply(lambda x: x[1] if x else np.nan)
no_match_trade_name["Best_Match_Index"] = no_match_trade_name["MatchTuple"].apply(lambda x: x[2] if x else np.nan)

CPU times: user 30.5 s, sys: 0 ns, total: 30.5 s
Wall time: 30.7 s


In [22]:
# We set the threshold 'Similarity_Score' to 86 because 50% of best drugs's 'Similarity_Score' are under 86. Also to reduce the human labor.
tmp_Fuzz_matched_names = no_match_trade_name[no_match_trade_name["Similarity_Score"] >= 90]

In [23]:
# Left merge df_ahfs_class_scdf to tmp_Fuzz_matched_names for catching 'AHFSClassNum' and 'AHFSClassText'
# Key: index
Fuzz_matched_names = pd.merge(
    tmp_Fuzz_matched_names[[
        'map_id',
        'otdate',
        'Source',
        'Number',
        'MedName',
        'DrugName',
        'DrugbankId',
        'Best_Match',
        'Similarity_Score',
        'Best_Match_Index']],
    df_ahfs_class_scdf[[
        'SCDF_CUI',
        'AHFSClassNum',
        'AHFSClassText']],
    left_on='Best_Match_Index',
    right_index= True,
    how='left')
print(f'Shape of Fuzz_matched_names: {Fuzz_matched_names.shape}')

Shape of Fuzz_matched_names: (305, 13)


In [ ]:
Fuzz_matched_names['DrugName'].nunique()

In [ ]:
# No matched after theFuzz matching (those Similarity_Score < 90)
Fuzz_no_matched_names = no_match_trade_name[no_match_trade_name["Similarity_Score"] < 90]
Fuzz_no_matched_names.shape

In [ ]:
Fuzz_no_matched_names['DrugName'].nunique()

# \*Drugs need Human Matching

- if 'RedCapMedName' is Vitmains/Supplements/OTCs: highlight and remove them
- if 'Best_Match' is not the correct name for 'RedCapMedName': identify them, and provide the correct 'SCDF_STR' from AHFS_SCDF.csv

In [ ]:
# Export to human matching
fp_no_matched = datetime.date.today().strftime('%Y%m%d') + '_Fuzz_no_matched_names.csv'
Fuzz_no_matched_names.to_csv(fp_cwd + fp_no_matched, index=False)

## Import the human matched Fuzz_no_matched_names

In [ ]:
df_human = pd.read_csv(fp_cwd + "Human_Matched_Fuzz_no_matched_names0212.csv", low_memory=False)

In [ ]:
# Exclude vitamins/supplements/otc drugs
tmp_df_human = df_human[df_human['Vit_Sup_OTC'] == 0].copy()

# Adjust formatting for ADRC & ARCHES/DRIVES
tmp_df_human = tmp_df_human.rename(columns={'RedCapMedName' : 'MedName'})

# Add 'otdate' and 'Source
fp_med_checkpoint = datetime.date.today().strftime('%Y%m%d') + '_CHECKPOINT_all_ADRC_ARCHES_DRIVES_meds.csv'
df_med = pd.read_csv(fp_cwd + fp_med_checkpoint, low_memory=False)
tmp_df_human = tmp_df_human.merge(
    df_med,
    how='left',
    on=['map_id', 'Number', 'MedName'])
tmp_df_human = tmp_df_human.drop(columns=['redcap_event_name'])

tmp_df_human.shape

In [ ]:
# Use human matched 'SCDF_STR' to replace wrong 'Best_Match', keeping the 'Best_Match' if no human annotation
tmp_df_human['Best_Match'] = tmp_df_human['SCDF_STR'].combine_first(tmp_df_human['Best_Match'])

In [ ]:
# Left merge to extract 'AHFSClassNum' and 'AHFSText'
merged_human_match = pd.merge(
    tmp_df_human,
    df_ahfs_class_scdf,
    left_on='Best_Match',
    right_on='SCDF_STR',
    how='left')

In [ ]:
# Only contains needed variables in the final matched_Fuzz_no_match_names 
matched_Fuzz_no_match_names = merged_human_match[[
    'map_id',
    #'redcap_event_name',
    'otdate',
    'Source',
    'Number',
    #'RedCapMedName',
    'MedName',
    'DrugName',
    'SCDF_CUI',
    'Best_Match',
    'AHFSClassNum',
    'AHFSClassText']]

# Match with UN

## 1. matched_ahfs_drug_class

In [ ]:
matched_ahfs_drug_class = pd.merge(
    matched_ahfs_drug,
    input_ahfs_class[[
        'AHFSClassID',
        'AHFSClassNum',
        'AHFSClassText',
        'AHFSRetired']].drop_duplicates(),
    on='AHFSClassID',
    how='left')
matched_ahfs_drug_class.shape

In [ ]:
# Check the NAs, ask DrugBank why the keys are missing
check = matched_ahfs_drug_class[matched_ahfs_drug_class['AHFSClassText'].isna()]
check['DrugName'].value_counts()

## 2. matched_ahfs_trade_name_class

In [ ]:
# Remove unneeded columns and rename UN
matched_ahfs_trade_name = matched_ahfs_trade_name.drop(columns=['GenDrugName', 'AHFSClassID', 'UN_y','PrintClass']).rename(columns={'UN_x':'UN'})
matched_ahfs_trade_name.shape

In [ ]:
# Left merge 'AHFSClassNum', 'AHFSClassText' to matched_ahfs_trade_name
merged_matched_ahfs_trade_name = pd.merge(
    matched_ahfs_trade_name,
    input_ahfs_drug[['UN', 'AHFSClassID']].drop_duplicates(),
    on='UN',
    how='left')
merged_matched_ahfs_trade_name.head()

In [ ]:
# Left merge 'AHFSClassID', 'AHFSClassNum', 'AHFSClassText' to merged_matched_ahfs_trade_name
matched_ahfs_trade_name_class = pd.merge(
    merged_matched_ahfs_trade_name,
    input_ahfs_class[[
        'AHFSClassID',
        'AHFSClassNum',
        'AHFSClassText',
        'AHFSRetired']].drop_duplicates(),
    on='AHFSClassID',
    how='left') 
matched_ahfs_trade_name_class.shape

In [ ]:
# Missing AHFS class number
matched_ahfs_trade_name_class [matched_ahfs_trade_name_class['AHFSClassNum'].isna()]

In [ ]:
matched_ahfs_drug_class.isna().sum()

In [ ]:
matched_ahfs_trade_name_class.isna().sum()

## Merge all matched names

In [ ]:
tmp_full_matched_name = pd.concat([
    matched_ahfs_drug_class, # Missing: 0 AHFSClassID; 136 AHFSClassNum
    matched_ahfs_trade_name_class,
    Fuzz_matched_names,
    matched_Fuzz_no_match_names]).drop_duplicates().reset_index(drop=True)

tmp_full_matched_name.shape

In [ ]:
# Create a new column to store the mapped names from 
tmp_full_matched_name['mapped_name'] = (
    tmp_full_matched_name['GenDrugName']
    .fillna(tmp_full_matched_name['Tradename'])
    .fillna(tmp_full_matched_name['Best_Match'])
)

In [ ]:
print(f'Unique classes of med names: {tmp_full_matched_name['AHFSClassText'].nunique()}')

## Merge all matched names to class extend

In [ ]:
tmp_full_matched_name_extend = pd.merge(
    tmp_full_matched_name,
    class_extend,
    on='AHFSClassNum',
    how='left')

In [ ]:
full_matched_name_extend = tmp_full_matched_name_extend[[
    'map_id',
    'otdate',
    'Source',
    'Number',
    'MedName',
    'mapped_name',
    'AHFSClassNum',
    'AHFSClassText',
    'AHFSClassText_1',
    'AHFSClassText_2',
    'AHFSClassText_3',
    'AHFSClassText_4']]
full_matched_name_extend.shape

In [ ]:
mapped_classes = full_matched_name_extend[['AHFSClassText']].drop_duplicates().reset_index(drop=True).dropna()

In [ ]:
# Remove those with following Vitamins, Supplements, and OTCs
full_matched_name_extend = full_matched_name_extend[~full_matched_name_extend['MedName'].str.contains(
    r'CoQ-10|melatonin|biotin|Milk|Lutein|Peppermint oil|Red Yeast Rice|Levemir Flex Pen|'
    r'Fiasp Flex Touch Injection|ENERGIZE|Oil|B-12|softener|Colace|Docusate|Q10|Enzyme|'
    r'Glucosamine|wax|Hydroxocobalamin|Zinc sulfate|Chondroitin sulfates|hyaluronate|'
    r'methylsulfonylmethane|Lactobacillus acidophilus|Melatonin / tryptophan|L-glutamine|lecithin|phenazopyridine|arginine Extended Release Oral Tablet',
    case=False, na=False
)].drop_duplicates()

In [ ]:
fp_full_matched_extend = datetime.date.today().strftime('%Y%m%d') + '_full_matched_name_extend.csv'
full_matched_name_extend.to_csv(fp_cwd + fp_full_matched_extend, index=False)

In [ ]:
# Sanity check
full_matched_name_extend ['mapped_name'].nunique()

# Total Execution Time

In [ ]:
analysis_end = timer()
print(datetime.timedelta(seconds=(analysis_end - analysis_start)))